# Ingrediënten klusteren
In de database staan heel veel ingredienten  die op elkaar lijken, maar net andere spelling hebben of in meervoud staan. Bijvoorbeeld 'zongedroogde tomaat' en 'zongedroogde tomaten' of 'olijf olie' en '(olijf) olie'. Daardoor kan er in twee recepten zowat hetzelfde ingrediënt zitten, maar toch op een andere plek een 1 of een 0 krijgen. Hier heb ik gewerkt aan code om dit soort gevallen te vinden en samen te voegen.

### Imports

In [28]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from IPython.display import display
import random
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score

### Data inlezen en bestuderen

In [29]:
import pandas as pd
from IPython.display import display

tagsFile = "tags"
ingredFile = "ingredients"

df_ingredients = pd.read_csv(f"/data/foodboost/{ingredFile}.csv")
df_tags = pd.read_csv(f"/data/foodboost/{tagsFile}.csv")

In [30]:
listoftags = df_tags['tag'].unique().tolist()
list_of_recipes = df_tags['recipe'].unique().tolist()
#tags = ['tags']

In [31]:
df_tags

,Unnamed: 0,recipe,tag
0,0,Kruidnoten met choco-discodip,hollands
1,1,Kruidnoten met choco-discodip,gebak
2,2,Kruidnoten met choco-discodip,gebak
3,3,Kruidnoten met choco-discodip,sinterklaas
4,4,Kruidnoten met choco-discodip,sinterklaasavond
...,...,...,...
46035,46035,Pittige truffels,nagerecht
46036,46036,Biefstuk met rodewijnsaus en ham,snel
46037,46037,Biefstuk met rodewijnsaus en ham,mediterraan
46038,46038,Biefstuk met rodewijnsaus en ham,hoofdgerecht


In [32]:
df_ingredients

,Unnamed: 0,recipe,ingredient,quantity,unit
0,0,Kruidnoten met choco-discodip,melkchocolade,100.0,g
1,1,Kruidnoten met choco-discodip,kruidnoten,100.0,g
2,2,Kruidnoten met choco-discodip,discodip,2.0,el
3,3,Kruidnoten in marsepein,blanke marsepein,150.0,g
4,4,Kruidnoten in marsepein,ongezouten roomboter,15.0,g
...,...,...,...,...,...
71804,71804,Biefstuk met rodewijnsaus en ham,olijfolie,4.0,el
71805,71805,Biefstuk met rodewijnsaus en ham,biefstukken,4.0,NaN
71806,71806,Biefstuk met rodewijnsaus en ham,boter,25.0,g
71807,71807,Biefstuk met rodewijnsaus en ham,serranoham,4.0,plakken


### Basis functies

In [33]:
# geeft een lijst van alle recepten die een specifieke tag bevatten
def recepten_bij_tag(tag):
    a = df_tags.loc[df_tags['tag'] == tag].recipe.to_list()
    return a

# geeft een lijst van de tags die aan een recept gelinked zijn
def tags_bij_recept(gerecht):
    a = df_tags.loc[df_tags['recipe'] == gerecht].tag.to_list()
    return a

In [35]:
# lijst met alle tags die bij verzameling recepten hoort.
def unieke_platte_lijst(gerechten_lijst):
    list_of_tags = []
    for i in gerechten_lijst:
        list_of_tags.append(tags_bij_recept(i))
    flatten = list(np.concatenate(list_of_tags).flat)
    flatten = np.unique(np.array(flatten))
    return flatten

In [37]:
# geeft een lijst van de ingrediënten die in een recept zitten
def ingredienten_bij_recept(gerecht):
    a = df_ingredients.loc[df_ingredients['recipe'] == gerecht].ingredient.to_list()
    return a

### Dataset krimpen
Omdat we in dit project alleen kijken naar avondeten recepten, heb ik hier de recepten met tags 'hoofdgerecht' en 'diner' geselecteerd voor de nieuwe dataset waar mee gewerkt wordt.

In [39]:
# alleen recepten met tag hoofdgerecht of diner selecteren
diner_set1 = set(recepten_bij_tag('hoofdgerecht')) 
diner_set2 = set(recepten_bij_tag('diner')) 
diner = diner_set1.union(diner_set2)

In [40]:
kleine_df = df_ingredients[df_ingredients['recipe'].isin(diner)]

In [41]:
list_of_ingredients = kleine_df['ingredient'].unique().tolist()

In [42]:
len(list_of_ingredients)

5006

Er zijn dus 5006 verschillende ingredienten in de dataset met alleen avondeten

### Groeperen en ingredienten count bekijken

In [43]:
teller = kleine_df.groupby(['ingredient']).recipe.count().to_frame()

In [44]:
teller

,recipe
ingredient,
(arachide)olie,80
(herten)biefstukken,1
(houdbare) melk,1
(magere) shoarmareepjes,1
(olijf)olie,107
...,...
zwarte-peperkorrel,1
zwarte-peperkorrels,5
zwarteolijvenplakjes,1


In [45]:
eenmalige_ingredients = teller[teller['recipe'] == 1]
eenmalige_ingredients

,recipe
ingredient,
(herten)biefstukken,1
(houdbare) melk,1
(magere) shoarmareepjes,1
(wilde) perzik,1
(witte) quinoa,1
...,...
zwarte sesamzaadje,1
zwarte tagliatelle,1
zwarte-peperkorrel,1


Je ziet dat er 2298 van de 5006 ingrediënten maar 1 keer voor komt, waaronder bijvoorbeeld '(houdbare) melk' en 'zwarte-peperkorrel'. Nou zullen er veel meer recepten zijn die deze ingrediënten bevatten, maar dan met een andere spelling. Daarom willen we deze vinden en de ingredienten die sterk op elkaar lijken als hetzelfde zien.

### Vergelijkbare Ingrediënten vinden
Tijdens de DataCamp courses heb ik geleerd te werken met de fuzzywuzzy library. Deze ga ik nu gebruiken om recepten te vinden die sterk overeenkomen. 

In [46]:
from fuzzywuzzy import fuzz, process

In [47]:
# Deze functie vind de string die t meest op meegegeven string lijken uit ingredienten.
def find_most_simular(string, limit):
    ingredient_lijst = df_ingredients['ingredient'].unique()
    return process.extract(string, ingredient_lijst, limit=limit)
    # process.extract kijkt in hoeveel stappen je van de ene string naar de andere kan. Hoe minder stappen nodig zijn, hoe hoger het percentage

In [48]:
find_most_simular('bloemkool', 25)

[('bloemkool', 100),
 ('bloem', 90),
 ('koelverse bloemkoolroosjes', 90),
 ('koelverse bloemkoolrijst', 90),
 ('koelverse bloemkool- en broccoliroosjes', 90),
 ('koelvers bloemkoolroosje', 90),
 ('Bonduelle diepvries bloemkoolrijst', 90),
 ('kleine bloemkool', 90),
 ('koelverse broccoli, wortel, bloemkool', 90),
 ('bloemkoolroosjes', 90),
 ('bloemkoolrijst', 90),
 ('bloemkool- en broccoliroosjes', 90),
 ('bloemkoolpizzabodem', 90),
 ('Fairtrade Original Libanese geroosterde bloemkool kruidenpasta', 90),
 ('Magioni bloemkoolpizzabodem', 90),
 ('AH Biologisch bloemkool', 90),
 ('koelverse minibloemkool en -broccoli', 90),
 ('ovengroente pompoen, zoete aardappel en bloemkool', 90),
 ('bloemkool &amp; broccoli', 90),
 ('diepvries bloemkoolroosjes', 90),
 ('mix voor gehakt met bloemkool', 90),
 ('boerenkool', 74),
 ('rodekool', 71),
 ('palmkool', 71),
 ('zonnebloemolie', 70)]

Dit voorbeeld hierboven (van bloemkool) heb ik voor meerdere ingredienten gedaan en gezien dat als het onder de 90 komt, het niet meer heel erg erop lijkt. Daar heb ik in de functie hieronder gewerkt met >= 90.

In [49]:
# functie lijst van alle simulars die 90 of meer overeenkomen
def find_simular_list(string):
    list_simulars = []
    ingredient_lijst = df_ingredients['ingredient'].unique()
    list_simulars1 = process.extract(string, ingredient_lijst, limit = df_ingredients.shape[0])
    for simular in list_simulars1:
        if simular[1] >= 90:
            list_simulars.append(simular)
    return list_simulars

In [50]:
find_simular_list('bloemkool') # geeft alle ingredienten met score 90 of hoger

[('bloemkool', 100),
 ('bloem', 90),
 ('koelverse bloemkoolroosjes', 90),
 ('koelverse bloemkoolrijst', 90),
 ('koelverse bloemkool- en broccoliroosjes', 90),
 ('koelvers bloemkoolroosje', 90),
 ('Bonduelle diepvries bloemkoolrijst', 90),
 ('kleine bloemkool', 90),
 ('koelverse broccoli, wortel, bloemkool', 90),
 ('bloemkoolroosjes', 90),
 ('bloemkoolrijst', 90),
 ('bloemkool- en broccoliroosjes', 90),
 ('bloemkoolpizzabodem', 90),
 ('Fairtrade Original Libanese geroosterde bloemkool kruidenpasta', 90),
 ('Magioni bloemkoolpizzabodem', 90),
 ('AH Biologisch bloemkool', 90),
 ('koelverse minibloemkool en -broccoli', 90),
 ('ovengroente pompoen, zoete aardappel en bloemkool', 90),
 ('bloemkool &amp; broccoli', 90),
 ('diepvries bloemkoolroosjes', 90),
 ('mix voor gehakt met bloemkool', 90)]

### Onderzoek naar ingredienten met count 1
Net hadden we in de df 'teller'gezien dat er veel ingredienten waren die 1 keer voor kwamen. Hieronder kijk ik welke van deze ingreidienten veel (meer dan 20) sterk verwante ingredienten hebben. Deze zijn relevant mee te nemen.

In [51]:
# functie die bepaald hoeveel simulars van 90% of hoger een ingredient heeft
def get_len_sims(i):
    string = eenmalige_ingredients.index[i]
    lijst_sims = find_simular_list(string)
    return len(lijst_sims)

In [53]:
gekozen_ingredients = []
for i in range(0, len(eenmalige_ingredients)):
    lengte = get_len_sims(i)
    if lengte >= 20:
        gekozen_ingredients.append(str(eenmalige_ingredients.index[i]))

In [55]:
gekozen_ingredients

['Italiaanse',
 'amandelen',
 'chili',
 'cola',
 'gerookte',
 'kippen',
 'kook',
 'kruidenmix',
 'linzen',
 'peper',
 'pure chocolade',
 'roomboter',
 'roomkaas',
 'rosé',
 'zalm']

# Conclusie
Uiteindelijk is dit bij het project Foodboost niet gebruikt, omdat wij ontdekte dat het niet heel veel toe zou voegen en op sommige punten het model juist ook slechter kan maken. Het schrijven van een functie die de dataset aanpast hebben wij daarom niet meer gedaan. Wel heb ik hiermee laten zien wat ik kan op het gebied van Data preprocessing.